# Knowledge density

We then predict the probability that individual, organization, or country, $s$ will increase its level of development in field $f$ by creating an indicator of the fraction of fields that are connected to field $f$ and that are already developed by $s$.

In [1]:
import sys
sys.path.append("../")
from lib.rs import research_space

In [2]:
key = "sjr"
rs = research_space(key)
rs.load(2011)
rs.set_institution("../../dataset/lattes/pesquisadores.csv", sep=";sep;")

In [3]:
rs.advantages()
rs.set_indicators()

### Prediction

Finally, to predict a transition of entity in field fbetween a pair of states (i.e. from inactive to active), we look at all fields that are in the initial state (i.e. inactive) and sort them by density ($\omega$sf).

*evaluating the transition from an inactive to an active state*

In [4]:
# Laender
rs.predict(9089204821424223, 'scientist', 'inactive-active')[:10]

[(0.32347, 'museology'),
 (0.30288, 'logic'),
 (0.30098, 'theoretical computer science'),
 (0.29712, 'signal processing'),
 (0.27709, 'human-computer interaction'),
 (0.25527, 'computer graphics and computer-aided design'),
 (0.22943, 'media technology'),
 (0.19965, 'control and systems engineering'),
 (0.18579, 'information systems and management'),
 (0.17672, 'nature and landscape conservation')]

In [5]:
# Ana
rs.predict(2408991231058279, 'scientist', 'inactive-active')[:10]

[(0.09604, 'media technology'),
 (0.08851, 'information systems'),
 (0.07919, 'theoretical computer science'),
 (0.07389, 'human-computer interaction'),
 (0.0667, 'computer science (miscellaneous)'),
 (0.0663, 'signal processing'),
 (0.06499, 'logic'),
 (0.06089, 'computer graphics and computer-aided design'),
 (0.05979, 'computer vision and pattern recognition'),
 (0.05901, 'artificial intelligence')]

In [11]:
# Heloisa Starling
rs.predict(3321652451642202, 'scientist', 'inactive-active')[:10]

[(0.10867, 'religious studies'),
 (0.09576, 'cultural studies'),
 (0.09082, 'political science and international relations'),
 (0.08657, 'gender studies'),
 (0.05983, 'anthropology'),
 (0.0576, 'demography'),
 (0.05637, 'literature and literary theory'),
 (0.05571, 'history and philosophy of science'),
 (0.05337, 'geography, planning and development'),
 (0.05285, 'economics, econometrics and finance (miscellaneous)')]

In [10]:
# Marcos Lisboa
rs.predict(1399818650600192, 'scientist', 'inactive-active')[:10]

[(0.10225, 'political science and international relations'),
 (0.08792, 'finance'),
 (0.05949, 'development'),
 (0.052, 'forestry'),
 (0.04907, 'accounting'),
 (0.0461, 'social sciences (miscellaneous)'),
 (0.04417, 'business, management and accounting (miscellaneous)'),
 (0.04264, 'sociology and political science'),
 (0.03148, 'public administration'),
 (0.03034, 'business and international management')]

### Accuracy